In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
# np.random.seed(710319)
n_1 = 300
n_2 = 300

rvs1 = stats.norm.rvs(size=n_1, loc=0, scale=1)
rvs2 = stats.norm.rvs(size=n_2, loc=0, scale=1)
stats.ks_2samp(rvs1, rvs2)

In [ ]:
ninst = 100
pvals_ens = []
scales = np.linspace(1, 1.5, 1000)
for _ in range(ninst):
    pvals = []
    for _scale in scales:
        rvs1 = stats.norm.rvs(size=n_1, loc=0, scale=1)
        rvs2 = stats.norm.rvs(size=n_2, loc=0, scale=_scale)
        _, p_val = stats.ks_2samp(rvs1, rvs2)
        pvals.append(p_val)
    pvals_ens.append(pvals)

In [ ]:
_, axis = plt.subplots(1, 1, figsize=(7, 5))
axis.plot(scales - 1.0, np.array(pvals_ens).mean(axis=0))
axis.axhline(0.05, color="k", ls="--")
axis.set_xlabel("Std Dev difference")
axis.set_ylabel("Ensemble mean p-value")
axis.set_title(f"K-S Test ensemble for {ninst} members")
axis.text(0.05, 0.055, "5%")
axis.grid(ls="--")
plt.savefig("ks_explore.png")